In [23]:
# imports
from collections import defaultdict, Counter


In [27]:
# utils
def get_input(day: int, test: bool = False):    
    with open(f"input/day{day}{'_test' if test else ''}.txt", "r") as f:
        return f.read()

def get_input_as_rows(day: int, test: bool = False):
    return get_input(day, test).split("\n")


In [28]:
# day 1
## part one
input_day1 = get_input_as_rows(1)
ll = [int(row.split(" ")[0].strip()) for row in input_day1]
rl = [int(row.split(" ")[-1].strip()) for row in input_day1]
ll.sort()
rl.sort()

total_distance = sum([abs(ll[i]- rl[i]) for i in range(len(ll))])
print(total_distance)

## part two
input_day1 = get_input_as_rows(1)
ll = [int(row.split(" ")[0].strip()) for row in input_day1]
rl = [int(row.split(" ")[-1].strip()) for row in input_day1]
c = Counter(rl)
similarity_score = sum([num * c[num] for num in ll])
print(similarity_score)


1830467
26674158
